In [16]:
%matplotlib inline
import sympy
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize
import math
from math import pi
np.set_printoptions(
    linewidth=120, formatter={
        'float': lambda x: f"{0:8.4g}" if abs(x) < 1e-10 else f"{x:8.4g}"})
np.random.seed(0)
from spatialmath import *
from spatialmath.base import *
from roboticstoolbox import *

CINEMATICA INVERSA PARA UN BRAZO ROBOTICO EN 2D


Solucion por formato cerrado con simbolos y algebra

Utilizando Sympy para definir constantes simbolicas para la longitud de los brazos del robot


In [17]:
a0, a1 = sympy.symbols("a0 a1")


Sistema de transformaciones elementales del robot 

In [67]:
e = ET2.R() * ET2.tx(a0) * ET2.R() * ET2.tx(a1)
e

R(q0) ⊕ tx(a0) ⊕ R(q1) ⊕ tx(a1)


Definir variables simbolicas para representar los angulos de las articulaciones

In [19]:
q0, q1 = sympy.symbols("q0 q1")

Calculando mediante cinematica directa la pose del efector final como funcion de las articulaciones variables

In [20]:
TE = e.fkine([q0, q1])
TE

  cos(q0 + q1) -sin(q0 + q1) a0*cos(q0) + a1*cos(q0 + q1)  
  sin(q0 + q1) cos(q0 + q1) a0*sin(q0) + a1*sin(q0 + q1)  
  0            0            1             


Trabajamos solo con la posicion del efector final en (x,y), ecuacion 1

In [71]:
x_fk, y_fk = TE.t
x_fk

a0*cos(q0) + a1*cos(q0 + q1)

In [72]:
y_fk


a0*sin(q0) + a1*sin(q0 + q1)

Definir dos variables simbolicas para la posicion final deseada del efector (x, y), ecuacion 2

In [73]:
x, y = sympy.symbols("x, y")

Hasta aca se tiene 2 ecuaciones

Usando sympy con algunos trucos para definir y resolver q0 y q1

In [23]:
eq1 = (x_fk**2 + y_fk**2-x**2-y**2).trigsimp()
eq1

a0**2 + 2*a0*a1*cos(q1) + a1**2 - x**2 - y**2

La solucion para eq1 es una lista (2 configuraciones diferentes de angulos pueden dar la misma posicion del del efector final)

In [74]:
q1_sol = sympy.solve(eq1, q1)
q1_sol

[-acos(-(a0**2 + a1**2 - x**2 - y**2)/(2*a0*a1)) + 2*pi,
 acos((-a0**2 - a1**2 + x**2 + y**2)/(2*a0*a1))]

In [82]:
type(q1_sol[1])

acos

Definicion de q0, expandimos la ecuacion

In [25]:
eq0 = tuple(map(sympy.expand_trig, [x_fk - x, y_fk - y]))
eq0

(a0*cos(q0) + a1*(-sin(q0)*sin(q1) + cos(q0)*cos(q1)) - x,
 a0*sin(q0) + a1*(sin(q0)*cos(q1) + sin(q1)*cos(q0)) - y)

Solucion de q0 en terminos de cos(q0) y sen(q0)

In [76]:
q0_sol = sympy.solve(eq0, [sympy.sin(q0), sympy.cos(q0)])
q0_sol

{sin(q0): a0*y/(a0**2 + 2*a0*a1*cos(q1) + a1**2*sin(q1)**2 + a1**2*cos(q1)**2) - a1*x*sin(q1)/(a0**2 + 2*a0*a1*cos(q1) + a1**2*sin(q1)**2 + a1**2*cos(q1)**2) + a1*y*cos(q1)/(a0**2 + 2*a0*a1*cos(q1) + a1**2*sin(q1)**2 + a1**2*cos(q1)**2),
 cos(q0): a0*x/(a0**2 + 2*a0*a1*cos(q1) + a1**2*sin(q1)**2 + a1**2*cos(q1)**2) + a1*x*cos(q1)/(a0**2 + 2*a0*a1*cos(q1) + a1**2*sin(q1)**2 + a1**2*cos(q1)**2) + a1*y*sin(q1)/(a0**2 + 2*a0*a1*cos(q1) + a1**2*sin(q1)**2 + a1**2*cos(q1)**2)}

In [80]:
type(q0_sol)

dict

La relacion del coseno y del seno nos da la tangente

In [77]:
q0_sol_2 = sympy.atan2(q0_sol[sympy.sin(q0)], q0_sol[sympy.cos(q0)]).simplify()
q0_sol_2


atan2((a0*y - a1*x*sin(q1) + a1*y*cos(q1))/(a0**2 + 2*a0*a1*cos(q1) + a1**2), (a0*x + a1*x*cos(q1) + a1*y*sin(q1))/(a0**2 + 2*a0*a1*cos(q1) + a1**2))

In [81]:
type(q0_sol_2)

atan2

Evaluacion para la ecuacion eq1, componente 1 de la lista de soluciones

In [89]:
q1_sol_subs = q1_sol[0].subs({x:1, y:1, a0:1, a1:1})
q1_sol_subs

3*pi/2

In [90]:
type(q1_sol_subs)

sympy.core.mul.Mul

Evaluacion para ecuacion eq1 como punto flotante y convierte la expresion simbolica en valores numericos

In [84]:
q1_num = q1_sol_subs.evalf()
q1_num

4.71238898038469

In [91]:
type(q1_num)

sympy.core.numbers.Float

In [92]:
type(float(q1_num))

float

Evaluacion para ecuacion eq0

In [96]:
q0_sol_subs = q0_sol_2.subs({x:1, y:1, a0:1, a1:1, q1:q1_num})
q0_sol_subs

-1.5707963267949 + pi

In [99]:
q0_num = q0_sol_subs.evalf()
q0_num

1.57079632679490

In [100]:
type(float(q0_num))

float